# 기본 SQL Agent 튜토리얼

이 노트북은 LangGraph를 사용하여 간단한 SQL Agent를 구현하는 기본적인 예제입니다.
Agent와 LangGraph의 기본기를 알고 있는 학생들을 위한 입문 수준의 SQL Agent입니다.

## 주요 학습 목표:
1. SQL 데이터베이스와 상호작용하는 기본적인 Agent 구조 이해
2. LangGraph를 사용한 간단한 워크플로우 구현
3. SQL 도구 사용법 및 오류 처리 기초

## 진행 순서:
1. 환경 설정 및 라이브러리 임포트
2. 데이터베이스 설정
3. Agent 상태 및 도구 정의
4. 기본 SQL Agent 구현
5. Agent 실행 및 테스트


## 1. 환경 설정 및 라이브러리 임포트

필요한 라이브러리들을 임포트하고 환경을 설정합니다.


In [1]:
import os
import requests
from typing import Annotated, Literal, TypedDict

from dotenv import load_dotenv

from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver

# 환경 변수 로드
load_dotenv()

print("✅ 라이브러리 임포트 완료!")


✅ 라이브러리 임포트 완료!


## 2. 데이터베이스 설정

Chinook 샘플 데이터베이스를 다운로드하고 SQLDatabase 인스턴스를 생성합니다.


In [2]:
def setup_database():
    """Chinook 샘플 데이터베이스를 다운로드하고 설정합니다."""
    
    # Chinook 데이터베이스 다운로드
    url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"
    
    if not os.path.exists("Chinook.db"):
        print("Chinook 데이터베이스를 다운로드 중...")
        response = requests.get(url)
        
        if response.status_code == 200:
            with open("Chinook.db", "wb") as file:
                file.write(response.content)
            print("데이터베이스 다운로드 완료!")
        else:
            raise Exception(f"데이터베이스 다운로드 실패: {response.status_code}")
    else:
        print("이미 데이터베이스 파일이 존재합니다.")
    
    # SQLDatabase 인스턴스 생성
    db = SQLDatabase.from_uri("sqlite:///Chinook.db")
    print(f"사용 가능한 테이블: {db.get_usable_table_names()}")
    
    return db

# 데이터베이스 설정 실행
db = setup_database()


이미 데이터베이스 파일이 존재합니다.
사용 가능한 테이블: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


## 3. Agent 상태 및 도구 정의

Agent의 상태와 SQL 쿼리 실행 도구를 정의합니다.


In [3]:
# Agent 상태 정의
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# SQL 쿼리 실행 도구 정의
@tool
def execute_sql_query(query: str) -> str:
    """
    SQL 쿼리를 실행하고 결과를 반환합니다.
    오류가 발생하면 오류 메시지를 반환합니다.
    """
    try:
        result = db.run(query)
        return str(result)
    except Exception as e:
        return f"쿼리 실행 오류: {str(e)}"

print("✅ 상태 및 도구 정의 완료!")


✅ 상태 및 도구 정의 완료!


## 4. 기본 SQL Agent 구현

LangGraph를 사용하여 기본적인 SQL Agent를 구현합니다.


In [4]:
def create_basic_sql_agent():
    """기본적인 SQL Agent를 생성합니다."""
    
    # LLM 초기화
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    # 도구 목록
    tools = [execute_sql_query]
    
    # 도구가 바인딩된 LLM
    llm_with_tools = llm.bind_tools(tools)
    
    # Agent 노드 정의
    def agent_node(state: State):
        """Agent가 사용자 질문을 분석하고 적절한 도구를 호출합니다."""
        
        # 시스템 메시지 추가
        system_message = """당신은 SQL 데이터베이스 전문가입니다.
        
사용자의 질문을 분석하여 적절한 SQL 쿼리를 작성하고 실행하세요.
        
데이터베이스 정보:
- Chinook 음악 스토어 데이터베이스
- 주요 테이블: Artist, Album, Track, Customer, Invoice, Employee 등

다음 단계를 따르세요:
1. 사용자 질문 분석
2. 적절한 SQL 쿼리 작성
3. execute_sql_query 도구를 사용하여 쿼리 실행
4. 결과를 사용자가 이해하기 쉽게 설명

주의사항:
- SELECT 쿼리만 사용하세요 (INSERT, UPDATE, DELETE 금지)
- 쿼리는 정확한 SQLite 문법을 사용하세요
"""
        
        messages = [AIMessage(content=system_message)] + state["messages"]
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # 도구 노드
    tool_node = ToolNode(tools)
    
    # 라우팅 함수
    def should_continue(state: State) -> Literal["tools", "end"]:
        """다음 단계를 결정합니다."""
        last_message = state["messages"][-1]
        
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            return "tools"
        else:
            return "end"
    
    # 그래프 생성
    workflow = StateGraph(State)
    
    # 노드 추가
    workflow.add_node("agent", agent_node)
    workflow.add_node("tools", tool_node)
    
    # 엣지 추가
    workflow.add_edge(START, "agent")
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "tools": "tools",
            "end": END
        }
    )
    workflow.add_edge("tools", "agent")
    
    # 그래프 컴파일
    app = workflow.compile(checkpointer=MemorySaver())
    
    return app

# Agent 생성
app = create_basic_sql_agent()
print("✅ 기본 SQL Agent 생성 완료!")


✅ 기본 SQL Agent 생성 완료!


## 5. Agent 실행 함수 정의

Agent를 실행하고 결과를 출력하는 함수를 정의합니다.


In [5]:
def run_basic_agent(app, question: str):
    """기본 SQL Agent를 실행합니다."""
    
    config = {"configurable": {"thread_id": "basic_sql_agent"}}
    
    inputs = {
        "messages": [HumanMessage(content=question)]
    }
    
    print(f"\n📋 질문: {question}")
    print("=" * 50)
    
    for output in app.stream(inputs, config):
        for key, value in output.items():
            print(f"\n🔄 {key}:")
            if "messages" in value:
                last_message = value["messages"][-1]
                if hasattr(last_message, 'content') and last_message.content:
                    print(f"내용: {last_message.content}")
                if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
                    for tool_call in last_message.tool_calls:
                        print(f"도구 호출: {tool_call['name']}")
                        print(f"쿼리: {tool_call['args']['query']}")

print("✅ 실행 함수 정의 완료!")


✅ 실행 함수 정의 완료!


## 6. 기본 테스트 실행

간단한 질문으로 Agent를 테스트해봅시다.


In [6]:
# 첫 번째 테스트: 테이블 목록 조회
run_basic_agent(app, "데이터베이스에 어떤 테이블들이 있나요?")



📋 질문: 데이터베이스에 어떤 테이블들이 있나요?

🔄 agent:
도구 호출: execute_sql_query
쿼리: SELECT name FROM sqlite_master WHERE type='table';

🔄 tools:
내용: [('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]

🔄 agent:
내용: 데이터베이스에는 다음과 같은 테이블들이 있습니다:

1. Album
2. Artist
3. Customer
4. Employee
5. Genre
6. Invoice
7. InvoiceLine
8. MediaType
9. Playlist
10. PlaylistTrack
11. Track

각 테이블은 음악 스토어와 관련된 다양한 정보를 저장하고 있습니다. 추가적인 질문이 있으시면 말씀해 주세요!


In [7]:
# 두 번째 테스트: 아티스트 조회
run_basic_agent(app, "가장 인기 있는 아티스트 10명을 보여주세요")



📋 질문: 가장 인기 있는 아티스트 10명을 보여주세요

🔄 agent:
도구 호출: execute_sql_query
쿼리: SELECT Artist.Name, COUNT(InvoiceLine.TrackId) AS SalesCount 
FROM Artist 
JOIN Album ON Artist.ArtistId = Album.ArtistId 
JOIN Track ON Album.AlbumId = Track.AlbumId 
JOIN InvoiceLine ON Track.TrackId = InvoiceLine.TrackId 
GROUP BY Artist.ArtistId 
ORDER BY SalesCount DESC 
LIMIT 10;

🔄 tools:
내용: [('Iron Maiden', 140), ('U2', 107), ('Metallica', 91), ('Led Zeppelin', 87), ('Os Paralamas Do Sucesso', 45), ('Deep Purple', 44), ('Faith No More', 42), ('Lost', 41), ('Eric Clapton', 40), ('R.E.M.', 39)]

🔄 agent:
내용: 가장 인기 있는 아티스트 10명은 다음과 같습니다:

1. **Iron Maiden** - 140회 판매
2. **U2** - 107회 판매
3. **Metallica** - 91회 판매
4. **Led Zeppelin** - 87회 판매
5. **Os Paralamas Do Sucesso** - 45회 판매
6. **Deep Purple** - 44회 판매
7. **Faith No More** - 42회 판매
8. **Lost** - 41회 판매
9. **Eric Clapton** - 40회 판매
10. **R.E.M.** - 39회 판매

이 아티스트들은 판매량 기준으로 가장 인기가 많습니다. 추가적인 정보가 필요하시면 말씀해 주세요!


In [ ]:
# 세 번째 테스트: 매출 조회
run_basic_agent(app, "2009년에 총 얼마의 매출이 있었나요?")


In [ ]:
# 네 번째 테스트: 국가별 고객 수
run_basic_agent(app, "각 국가별 고객 수를 알려주세요")


## 7. 추가 실습

### 실습 문제들
아래 질문들로 추가 실습을 해보세요!


In [ ]:
# 추가 실습 문제들을 직접 실행해보세요!

practice_questions = [
    "Led Zeppelin의 앨범을 모두 보여주세요",
    "가장 비싼 음악 트랙 5개는 무엇인가요?",
    "직원 중에 누가 가장 많은 고객을 담당하고 있나요?",
    "Rock 장르의 음악이 몇 개나 있나요?"
]

# 원하는 질문을 선택해서 실행해보세요
# 예시:
# run_basic_agent(app, practice_questions[0])

print("🎯 실습 문제 목록:")
for i, question in enumerate(practice_questions, 1):
    print(f"{i}. {question}")
print("\n위의 질문들 중 하나를 선택해서 run_basic_agent(app, practice_questions[인덱스])로 실행해보세요!")


## 8. 마무리 및 다음 단계

### 이 튜토리얼에서 배운 것들:
1. ✅ SQL 데이터베이스와 상호작용하는 기본적인 Agent 구조
2. ✅ LangGraph를 사용한 간단한 워크플로우 구현  
3. ✅ SQL 도구 사용법 및 기본적인 오류 처리
4. ✅ Agent 실행 및 테스트 방법

### 다음 학습 단계:
- `02_advanced_sql_agent.ipynb`: 실무 수준의 고급 SQL Agent 학습
- 복잡한 워크플로우와 향상된 오류 처리 방법
- 쿼리 검증 시스템 및 성능 모니터링

### 주요 개선점들:
- 더 정교한 상태 관리
- 단계별 워크플로우 (테이블 → 스키마 → 쿼리 → 검증 → 실행)
- 보안 강화 및 성능 최적화

축하합니다! 🎉 기본 SQL Agent를 성공적으로 구현했습니다!


## 7. 추가 실습 및 커스터마이징

### 실습 문제
아래 질문들로 추가 실습을 해보세요:

1. "Led Zeppelin의 앨범을 모두 보여주세요"
2. "가장 비싼 음악 트랙 5개는 무엇인가요?"
3. "직원 중에 누가 가장 많은 고객을 담당하고 있나요?"
4. "Rock 장르의 음악이 몇 개나 있나요?"

### 커스터마이징 아이디어
- 다른 데이터베이스 연결
- 추가 도구 구현 (차트 생성, 데이터 시각화 등)
- 더 정교한 오류 처리
- 쿼리 결과 포맷팅 개선
